# Decision analysis with Julia `Agents.jl` using Petitti, 2000, 2e. - Chapter 2 
Tomás Aragón, Updated 2026-01-21

This Julia Jupyter Notebook demonstrates decision analysis using agent-based (ABM) modeling with the `Agents.jl` Julia package. The example is based on the decision tree from Diana B. Petitti. *Meta-Analysis, Decision Analysis, and Cost-Effectiveness Analysis: Methods for Quantitative Synthesis in Medicine*. 2nd ed. Monographs in Epidemiology and Biostatistics, v. 31. Oxford University Press, 2000. https://doi.org/10.1093/acprof:oso/9780195133646.001.0001.

The ABM replicates these previous decision analyses:
- DA using a decision tree (`rdecision`): [R Jupyter Notebook (Part 2)](https://github.com/tomasaragon/di4h/blob/main/nb/Petitti2000/NB_R_rdecision_part2.ipynb)   
- DA using influence diagrams (`DecisionProgramming.jl`): [Julia Jupyter Notebook](https://github.com/tomasaragon/di4h/blob/main/nb/Petitti2000/NB_Julia_DecisionProgramming-jl.ipynb)

The `Agents.jl` package documentation is here: https://juliadynamics.github.io/Agents.jl/stable/.

Here is the decision tree from Petitti, 2000, 2e, Chapter 2, Figure 2-6 (FIGURE 1). 

<figure>
<img src="Petitti_2000_2e_ch02_fig2-6.png" width="800" alt="Petitti 2000 Figure 2-6"/>
<figcaption>FIGURE 1. Decision tree from Petitti, 2000, 2e, Chapter 2, Figure 2-6.</figcaption>
</figure>

Now, here is the decision tree as an influence diagram (FIGURE 2). Notice there is no arrow from the measles node to the value node. The value node only contains input information on revaccination status and mortality based on the scenario. If measles morbidity had been included in the scenario then we would have added an arrow from the measles node to value node. I find influence diagrams more concise and interpretable. Because influence diagrams are directed acyclic graphs (DAGs), they promote causal reasoning. 

<figure>
<img src="img_measles_decision_network_drawio.png" width="600" alt="Petitti 2000 Figure 2-1"/>
<figcaption>FIGURE 2: Influence diagram for Figure 2-6 from Petitti, 2000, 2e, Chapter 2.</figcaption> 
</figure>

We have a human agent that can experience these states: revaccination, exposure to measles, measles infection, and death from measles.  
- $P(E = 1) = 0.20$
- $P(M = 1 \mid E = 1, R = 1) = 0.05$
- $P(M = 1 \mid E = 1, R = 0) = 0.33$
- $P(D = 1 \mid M = 1) = 0.0023$

where: $E$ is exposure to measles $\{1=yes, 0=no\}$, $M$ is measles infection $\{1=yes, 0=no\}$, $D$ is death from measles $\{1=yes, 0=no\}$, and $R$ is revaccination status $\{1=yes, 0=no\}$.

Steps to implement the agent-based model: 
1. create agent type
2. create agent stepping function
3. intitialize model: create model and add agents
4. run simulation
5. analyze results

In [9]:
#import Pkg # Run to update Agents.jl to v6+
## 1. Activate a new folder for this project
#Pkg.activate("DAProject");
## 2. Force install the latest version
#Pkg.add("Agents");
## 3. Check the version (should be v6+)
#Pkg.status("Agents");

In [10]:
using Agents
using Random
using Statistics

# Probabilities
const p_E1 = 0.20          
const p_M1_E1_R1 = 0.05    
const p_M1_E1_R0 = 0.33    
const p_D1_M1 = 0.0023     

# Utilities
const U_Alive = 1.0
const U_Dead = 0.0

0.0

In [11]:
@agent struct Patient(NoSpaceAgent)
    revaccinate::Bool 
    exposed::Bool     
    measles::Bool     
    dead::Bool        
    utility::Float64  
end

In [12]:
function agent_step!(agent::Patient, model)
    # 1. Chance Node E: Exposure
    # Use abmrng(model) to access the random number generator
    rng = abmrng(model)
    
    if rand(rng) < p_E1
        agent.exposed = true
        
        # 2. Chance Node M: Measles
        prob_measles = agent.revaccinate ? p_M1_E1_R1 : p_M1_E1_R0
        
        if rand(rng) < prob_measles
            agent.measles = true
            
            # 3. Chance Node D: Death
            if rand(rng) < p_D1_M1
                agent.dead = true
                agent.utility = U_Dead
            else
                agent.dead = false
                agent.utility = U_Alive
            end
        else
            agent.measles = false
            agent.utility = U_Alive
        end
    else
        agent.exposed = false
        agent.utility = U_Alive
    end
end

agent_step! (generic function with 1 method)

In [13]:
function initialize_model(; n_agents = 100_000, decision_revaccinate = true)
    space = nothing
    # Create model with v6 syntax
    model = StandardABM(Patient, space; rng = MersenneTwister(42), agent_step! = agent_step!)
    
    # Create agents
    for i in 1:n_agents
        add_agent!(model, decision_revaccinate, false, false, false, 0.0)
    end
    
    return model
end

initialize_model (generic function with 1 method)

In [14]:
# --- Scenario 1: R1 (Revaccinate) ---
model_R1 = initialize_model(n_agents = 1_000_000, decision_revaccinate = true)
run!(model_R1, 1) 

eu_R1 = mean(model_R1[id].utility for id in allids(model_R1))

# --- Scenario 2: R0 (Do Not Revaccinate) ---
model_R0 = initialize_model(n_agents = 1_000_000, decision_revaccinate = false)
run!(model_R0, 1) 

eu_R0 = mean(model_R0[id].utility for id in allids(model_R0))

# --- Results ---
println("--- Agents.jl Monte Carlo Results ---")
println("R1 (Revaccinate) Expected Utility: $eu_R1")
println("R0 (No Revax)    Expected Utility: $eu_R0")

--- Agents.jl Monte Carlo Results ---
R1 (Revaccinate) Expected Utility: 0.999968
R0 (No Revax)    Expected Utility: 0.999843


In [15]:
# calculate risk difference as on pp. 27-28
risk_death_no_revax = 1 - eu_R0
risk_death_revax = 1 - eu_R1
risk_difference = risk_death_no_revax - risk_death_revax
round(risk_difference, digits = 6)


0.000125

From Petitti, pp. 27-28: "The difference in the expected probability of death from measles between a strategy of revaccination and a strategy of no-revaccination is `0.000125`. This is interpreted to mean that 12.9 deaths from measles are prevented per 100,000 children revaccinated.

## Appendix 

In [16]:
methods(Agents.StandardABM)

# 4 methods for type constructor:
 [1] StandardABM(agents::C, agent_step::G, model_step::K, space::S, scheduler::F, properties::P, rng::R, agents_types::T, agents_first::Bool, maxid::Base.RefValue{Int64}, time::Base.RefValue{Int64}) where {S<:Union{Nothing, Agents.AbstractSpace}, A<:AbstractAgent, C<:Union{AbstractDict{Int64, A}, AbstractVector{A}}, T, G, K, F, P, R<:AbstractRNG}
     @ ~/.julia/packages/Agents/WuWeG/src/core/model_standard.jl:12
 [2] StandardABM(::Type{A}; ...) where A<:AbstractAgent
     @ ~/.julia/packages/Agents/WuWeG/src/core/model_standard.jl:131
 [3] StandardABM(::Type{A}, space::S; agent_step!, model_step!, container, scheduler, properties, rng, agents_first, warn, warn_deprecation) where {A<:AbstractAgent, S<:Union{Nothing, Agents.AbstractSpace}}
     @ ~/.julia/packages/Agents/WuWeG/src/core/model_standard.jl:131
 [4] StandardABM(agent::AbstractAgent, args::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/core/model_standard.jl:164

In [17]:
methods(add_agent!)

# 13 methods for generic function "add_agent!" from Agents:
  [1] add_agent!(A::Type{<:AbstractAgent}, model::AgentBasedModel{S}, properties::Vararg{Any, N}; kwargs...) where {N, S<:ContinuousSpace}
     @ ~/.julia/packages/Agents/WuWeG/src/spaces/continuous.jl:124
  [2] add_agent!(::AbstractAgent, ::AgentBasedModel, ::AgentBasedModel)
     @ ~/.julia/packages/Agents/WuWeG/src/ambiguities.jl:5
  [3] add_agent!(::AbstractAgent, ::Union{Function, Type}, ::AgentBasedModel, ::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/ambiguities.jl:4
  [4] add_agent!(agent::AbstractAgent, model::AgentBasedModel{Nothing})
     @ ~/.julia/packages/Agents/WuWeG/src/spaces/nothing.jl:20
  [5] add_agent!(agent::AbstractAgent, model::AgentBasedModel)
     @ ~/.julia/packages/Agents/WuWeG/src/core/space_interaction_API.jl:215
  [6] add_agent!(agent::AbstractAgent, pos, model::AgentBasedModel)
     @ ~/.julia/packages/Agents/WuWeG/src/core/space_interaction_API.jl:220
  [7] add_agent!(::AgentBasedModel, ::Union{Function, Type}, ::AgentBasedModel, ::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/ambiguities.jl:6
  [8] add_agent!(::AgentBasedModel, ::AgentBasedModel, ::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/ambiguities.jl:7
  [9] add_agent!(model::AgentBasedModel, args::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/core/space_interaction_API.jl:260
 [10] add_agent!(A::Union{Function, Type}, model::AgentBasedModel{Nothing}, args::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/spaces/nothing.jl:10
 [11] add_agent!(A::Union{Function, Type}, model::AgentBasedModel, args::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/core/space_interaction_API.jl:265
 [12] add_agent!(pos, A::Union{Function, Type}, model::AgentBasedModel, args::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/core/space_interaction_API.jl:281
 [13] add_agent!(pos, model::AgentBasedModel, args::Vararg{Any, N}; kwargs...) where N
     @ ~/.julia/packages/Agents/WuWeG/src/core/space_interaction_API.jl:270